In [5]:
import pandas as pd
autentica = pd.read_excel('ls.xlsx')

#adicionar senhas em autentica
senha = {'login': '2418048', 'senha': '19230'}
# autentica = autentica.append(senha, ignore_index=True)
display(autentica)


login = str(autentica['login'][2])
senha = str(autentica['senha'][2])

indisponivel = "indisponivel"

,login,senha
0,2184579,sis1986%
1,2184708,aeiou12345
2,2418048,19230


In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys(senha)

    nav.find_element_by_xpath('//*[@id="ok"]').click()

In [7]:
df_elemento_despesa = pd.DataFrame()
df_temp = pd.DataFrame(columns=('value', 'ed'))
df_temp.loc[0] = ['14', 'esportes']
# display(df_temp)

opcoes = []
valores = []

def varrer_ed():
#     abrir_suap()
    nav.get("https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/")
        
    valor_ed = nav.find_element_by_id('id_elemento_despesa')
    options = [x for x in valor_ed.find_elements_by_tag_name('option')]
    
    for i in options:
        # print(i.get_attribute('value')+" "+i.text)
        opcoes.append(i.text)
        valores.append(i.get_attribute('value'))
#         print(f'{i.text}-{i.get_attribute("value")}')

    df_elemento_despesa['value'] = valores
    df_elemento_despesa['ed']= opcoes
    
#     display(df_elemento_despesa)

In [8]:
def get_qtd():
    
    abrir_suap()
    
    df_tabela = pd.DataFrame()
    tabela = []
    codigos = []
    descricoes = []
    quantidades = []
    valores = []
    elemento = []
    
    try:
        pag = 1
        saida = True
        while saida:
                
            nav.get(f'https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/?page={pag}&material=&elemento_despesa=&estoque_tipo=emcampus&unidade_medida=')
            celula = nav.find_element_by_tag_name('td')
                
            if celula:
                lista = nav.find_element_by_tag_name('table')
                td = lista.find_elements_by_tag_name('td')
                
                for i in td:
                    tabela.append(i.text)
                
                pag += 1
            else:
                saida = False
                
    except NoSuchElementException:
        print('acabaram as páginas')
        
    for cod in range(0, len(tabela), 7):
        codigos.append(tabela[cod])

    for des in range(1, len(tabela), 7):
        descricoes.append(tabela[des])

    for qtd in range(4, len(tabela), 7):
        quantidades.append(tabela[qtd])
    
    for val in range(5, len(tabela), 7):
        valores.append(tabela[val])
        
    for ed in range(3, len(tabela), 7):
        elemento.append(tabela[ed])

    df_tabela['codigo'] = codigos
    df_tabela['descricao'] = descricoes
    
    df_tabela['qtd'] = quantidades
    df_tabela['qtd'] = pd.to_numeric(df_tabela['qtd'])
    
    df_tabela['vlr_total'] = valores
    df_tabela['vlr_total'] = df_tabela['vlr_total'].str[2:]
    df_tabela['vlr_total'] = pd.to_numeric(df_tabela['vlr_total'])
    
    df_tabela['vlr_unt'] = df_tabela['vlr_total'] / df_tabela['qtd']
    df_tabela['vlr_unt'] = df_tabela['vlr_unt'].round(2)
    
    df_tabela['ed'] = elemento
    
    df_tabela.to_excel('relatorio/relatorio.xlsx', index=False)
    display(df_tabela)
    
    nav.quit()


In [9]:
get_qtd()

acabaram as páginas


,codigo,descricao,qtd,vlr_total,vlr_unt,ed
0,30.04.1,"GLP - 13KG, GÁS LIQUEFEITO DE PETRÓLEO, PROPAN...",39,3198.00,82.00,04 - GAS E OUTROS MATERIAIS ENGARRAFADOS
1,30.06.100,RAÇÃO ANIMAL BALANCEADA PARA FRANGO CRESCIMENT...,1160,3271.20,2.82,06 - ALIMENTOS PARA ANIMAIS
2,30.06.101,"RAÇÃO ANIMAL BALANCEADA PARA BOVINO BEZERRO, S...",2680,6337.20,2.36,06 - ALIMENTOS PARA ANIMAIS
3,30.06.102,"RAÇÃO ANIMAL BALANCEADA PARA BOVINO LACTAÇÃO, ...",720,2052.00,2.85,06 - ALIMENTOS PARA ANIMAIS
4,30.06.107,"RAÇÃO ANIMAL BALANCEADA PARA SUÍNO INICIAL, SA...",280,733.60,2.62,06 - ALIMENTOS PARA ANIMAIS
...,...,...,...,...,...,...
637,30.42.960,"Alicate universal, 8, fabricado em aço cromo-v...",3,58.14,19.38,42 - FERRAMENTAS
638,30.44.66,"Placa de identificação, material PVC, 17cm x 1...",10,215.00,21.50,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
639,30.44.67,"Fita adesiva, material propileno, 20mm x 30m, ...",97,2163.10,22.30,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
640,30.44.68,"Fita adesiva, material PVC, tipo auto-adesiva ...",5,278.40,55.68,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
